In [5]:
import pandas as pd
import numpy as np

In [ ]:
class TrendDetection:
    def __init__(self, df, config):
        self.df = df.copy()
        self.short_period = config['short_period']
        self.medium_period = config['medium_period']
        self.long_period = config['long_period']
        self.backcandles = config['backcandles']
        self.threshold = config['threshold']
        # ma_trend
        self.df['trend'] = self.df.apply(self.ma_trend, axis=1)
        # ADX trend
        self.df['ADX_trend'] = self.adx_trend()
        # Categories for specified moving averages
        self.ma_periods = [self.short_period, self.medium_period, self.long_period]
        for period in self.ma_periods:
            ma_column = f'sma_{period}'
            category_column = f'category_sma_{period}'
            confirmed_signal_column = f'confirmed_signal_{period}'
            
            self.df[category_column] = self.check_candles(ma_column)
            self.df[confirmed_signal_column] = self.df.apply(lambda row: self.trend_confirmation(row, category_column), axis=1)
    
        
    def ma_trend(self, row):
        if row[f'sma_{self.short_period}'] > row[f'sma_{self.medium_period}'] > row[f'sma_{self.long_period}']:
            return 2  # uptrend
        elif row[f'sma_{self.short_period}'] < row[f'sma_{self.medium_period}'] < row[f'sma_{self.long_period}']:
            return 1  # downtrend
        else:
            return 0  # no trend
    
    def check_candles(self, ma_column):
        """Check the number of candles thaat are above or below the moving average curve
        to determine the trend
        Args:
            df (pd.DataFrame): The dataframe containing the stock data
            ma_column (str): The column name of the moving average curve"""
        category = []
        for i in range(self.backcandles, len(self.df)):
            if all(self.df['close'][i-self.backcandles:i] > self.df[ma_column][i-self.backcandles:i]):
                category.append('2') # uptrend
            elif all(self.df['close'][i-self.backcandles:i] < self.df[ma_column][i-self.backcandles:i]):
                category.append('1') # downtrend
            else:
                category.append('0') # no trend
        return category
    
    
    def adx_trend(self):
        trend_signal = []
        for i in range(len(self.df)):
            if self.df['ADX'][i] > self.threshold:
                if self.df['ADX_pos'][i] > self.df['ADX_neg'][i]:
                    trend_signal.append('2') # uptrend
                else:
                    trend_signal.append('1') # downtrend
            else:
                trend_signal.append('0') # no trend 
        return pd.Series(trend_signal)
    
    def trend_confirmation(self,row, ma_column):
        if row['ADX_trend'] == row[ma_column]:
            return row['ADX_trend']
        else:
            return 0

In [ ]:
class MomentumDetector

In [ ]:
# def supertrend(data, n, m):
    
#     """Calculates supertrend given historical data
#     n = n day ATR
#     m = multiplier"""
#     df = data.copy()
#     df["B-U"]=((df['high']+df['low'])/2) + m*df['ATR'] 
#     df["B-L"]=((df['high']+df['low'])/2) - m*df['ATR']
#     df["U-B"]=df["B-U"]
#     df["L-B"]=df["B-L"]
#     ind = df.index
#     for i in range(n,len(df)):
#         if df['close'][i-1]<=df['U-B'][i-1]:
#             df.loc[ind[i],'U-B']=min(df['B-U'][i],df['U-B'][i-1])
#         else:
#             df.loc[ind[i],'U-B']=df['B-U'][i]    
#     for i in range(n,len(df)):
#         if df['close'][i-1]>=df['L-B'][i-1]:
#             df.loc[ind[i],'L-B']=max(df['B-L'][i],df['L-B'][i-1])
#         else:
#             df.loc[ind[i],'L-B']=df['B-L'][i]  
#     df['Strend']=np.nan
#     for test in range(n,len(df)):
#         if df['close'][test-1]<=df['U-B'][test-1] and df['close'][test]>df['U-B'][test]:
#             df.loc[ind[test],'Strend']=df['L-B'][test]
#             break
#         if df['close'][test-1]>=df['L-B'][test-1] and df['close'][test]<df['L-B'][test]:
#             df.loc[ind[test],'Strend']=df['U-B'][test]
#             break
#     for i in range(test+1,len(df)):
#         if df['Strend'][i-1]==df['U-B'][i-1] and df['close'][i]<=df['U-B'][i]:
#             df.loc[ind[i],'Strend']=df['U-B'][i]
#         elif  df['Strend'][i-1]==df['U-B'][i-1] and df['close'][i]>=df['U-B'][i]:
#             df.loc[ind[i],'Strend']=df['L-B'][i]
#         elif df['Strend'][i-1]==df['L-B'][i-1] and df['close'][i]>=df['L-B'][i]:
#             df.loc[ind[i],'Strend']=df['L-B'][i]
#         elif df['Strend'][i-1]==df['L-B'][i-1] and df['close'][i]<=df['L-B'][i]:
#             df.loc[ind[i],'Strend']=df['U-B'][i]
#     return df['Strend']
    